In [12]:
import os, glob, sys
import pandas as pd
import geopandas as gpd
import fiona
from shapely.geometry import Polygon, mapping
import matplotlib.pyplot as plt
import folium
import rasterio
import rasterio.mask
import rasterio.plot
import numpy as np

pd.set_option('display.max_columns', None)

sys.path.insert(0, '/export/miro/ndeffense/LBRAT2104/GIT/eo-toolbox/tools/')

import folium_def
import remove_overlap

# Input / Output

## Input

In [2]:
path = '/export/projects/FAO-EOStat-Senegal/'

gps_xlsx = f'{path}GPS/base_nioro.xlsx'

odk_crop_csv     = f'{path}ODK/FormSenegal05.csv'
odk_non_crop_csv = f'{path}ODK/FormSenegal07.csv'

odk_shp = f'{path}ODK/polygons_ODK.shp'
roi_shp = f'{path}ROI/Senegal_DepartementNioro.shp'

lut_crop_csv     = f'{path}LUT/LUT_crop.csv'
lut_non_crop_csv = f'{path}LUT/LUT_non_crop.csv'

## Output

In [3]:
gps_shp_output          = f'{path}Output_Nico/gps_with_data.shp'
odk_crop_shp_output     = f'{path}Output_Nico/odk_crop_with_data.shp'
odk_non_crop_shp_output = f'{path}Output_Nico/odk_non_crop_with_data.shp'

full_shp_output = f'{path}Sen4Stat/IN_SITU/SITE_41/SEN_2021_SITE_41_ori.shp'

map_html_output = f'{path}Output_Nico/odk_gps_with_data.html'


# Parameters

In [4]:
buf_size = -10  # meter
min_area = 100  # square meter

# Open Look-up-Table

In [20]:
lut_crop_df = pd.read_csv(lut_crop_csv)

lut_crop_df.head()

,crop_code,crop_name
0,1,Arachide
1,2,Aubergine
2,3,Béréf
3,4,Bissap
4,5,Coton


In [21]:
lut_non_crop_df = pd.read_csv(lut_non_crop_csv)

lut_non_crop_df.head()

,lc_code,lc_name
0,1,Prairie
1,2,Jachère
2,3,Broussailes
3,4,Forêt
4,5,Sol nu


# Geometries : retrieval & cleaning

## 1. From GPS

### 1.1. Extraction from .gpx files into a GeoDataFrame

In [5]:
dict_gpx_list = []

wrong_pid = 0

for gpx_file in glob.glob(f'{path}GPS/*/*.gpx'):

    filename = os.path.basename(gpx_file)

    pid = filename[filename.find("Piste")+5:-4].lstrip('_ ')
    pid = pid.replace(' ','-')
    pid = pid.replace('---','-')
    pid = pid.replace('--','-')
    pid = pid.split('-')

    if len(pid) <= 5:
        pid = ''.join(pid)

        layer = fiona.open(gpx_file, layer='tracks')
        crs = layer.crs

        if crs['init'] == 'epsg:4326':

            dict_gpx = {'id': pid,
                        'coordinates': layer[0]['geometry']['coordinates'][0]}

            dict_gpx_list.append(dict_gpx)
    else:
        wrong_pid += 1


df = pd.DataFrame.from_dict(dict_gpx_list).drop_duplicates('id')

print(f'Good PID (whitout duplicates) : {len(df)}')
print(f'Wrong PID : {wrong_pid}')


df['geometry'] = df.coordinates.apply(Polygon)

df = df.drop('coordinates', axis=1)

gps_gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry='geometry').to_crs(epsg=32628)

gps_gdf.head()

Good PID (whitout duplicates) : 335
Wrong PID : 36


,id,geometry
0,62201022411,"POLYGON ((429548.498 1516286.092, 429548.825 1..."
1,62201022412,"POLYGON ((429863.904 1516202.422, 429863.342 1..."
2,62201022413,"POLYGON ((429972.245 1516435.077, 429973.528 1..."
3,62201022414,"POLYGON ((429974.731 1516287.245, 429976.051 1..."
4,62201022164,"POLYGON ((429808.854 1515355.640, 429808.746 1..."


### 1.2. Cleaning polygons

- Remove overlapping parts of polygons
- Apply negative buffer
- Remove too small polygons

In [6]:
print(f'Initial polygons : {len(gps_gdf)}')

gps_gdf = remove_overlap.remove_overlap(gps_gdf)

print(f'After removing overlapping parts of polygons : {len(gps_gdf)}')

gps_gdf['geometry'] = gps_gdf.buffer(buf_size)

print(f'After adding negative buffer of {buf_size}m : {len(gps_gdf)}')

gps_gdf['area'] = gps_gdf['geometry'].area.round(2)

gps_gdf = gps_gdf.loc[gps_gdf['area'] >= min_area]

print(f'After removing polygons where area is less than {min_area} square meters : {len(gps_gdf)}')

gps_gdf['collect'] = 'GPS'


print(f'--> There are {len(gps_gdf)} polygons obtained with the GPS')
gps_gdf.head()


Initial polygons : 335
After removing overlapping parts of polygons : 335
After adding negative buffer of -10m : 335
After removing polygons where area is less than 100 square meters : 324
--> There are 324 polygons obtained with the GPS


,id,geometry,area,collect
0,62201022411,"POLYGON ((429457.856 1516229.002, 429465.169 1...",5172.41,GPS
1,62201022412,"POLYGON ((429828.180 1516150.340, 429839.170 1...",12114.17,GPS
2,62201022413,"POLYGON ((429883.899 1516457.550, 429884.205 1...",6209.42,GPS
3,62201022414,"POLYGON ((429986.960 1516282.075, 429987.433 1...",2731.43,GPS
4,62201022164,"POLYGON ((429722.291 1515343.246, 429740.288 1...",1860.82,GPS


## 2. From ODK (tablet) - cropland

### 2.1. Read polygons from shapefile obtain with QGIS plugin

In [36]:
odk_gdf = gpd.read_file(odk_shp)

odk_gdf['geometry'] = [Polygon(mapping(x)['coordinates']) for x in odk_gdf.geometry]

odk_gdf = odk_gdf.to_crs(epsg=32628)

# Build ODK ID based on other fields

#odk_gdf['id'] = odk_gdf['id-Commune'].astype(str) + odk_gdf['id-Distric'].astype(str) + odk_gdf['id-Concess'].astype(str) + odk_gdf['id-Menage'].astype(str) + odk_gdf['id-Parcell'].astype(str)

odk_gdf['id'] = odk_gdf['meta-ins_1'].str.replace('_', '').str.replace(' ', '')

odk_gdf = odk_gdf.rename(columns={"Informat_1": "crop_code"})

odk_gdf = odk_gdf[['id','crop_code','geometry']]

odk_gdf.head()

,id,crop_code,geometry
0,6220201174341,1.0,"POLYGON ((393929.276 1528886.677, 393909.441 1..."
1,6220201173222,13.0,"POLYGON ((394090.930 1528320.745, 394039.179 1..."
2,6220201171611,1.0,"POLYGON ((394059.235 1529548.963, 394198.230 1..."
3,6220201171631,1.0,"POLYGON ((393936.493 1529548.371, 394068.694 1..."
4,6220201171661,1.0,"POLYGON ((393929.236 1529486.680, 394057.462 1..."


### 2.2. Cleaning polygons

- Remove duplicates ID
- Remove overlapping parts of polygons
- Apply negative buffer
- Remove too small polygons

In [42]:
print(f'Initial polygons : {len(odk_gdf)}')

odk_gdf = odk_gdf.drop_duplicates(subset=['id'])

print(f'After removing duplicates ID : {len(odk_gdf)}')

odk_gdf = remove_overlap.remove_overlap(odk_gdf)

print(f'After removing overlapping parts of polygons : {len(odk_gdf)}')

odk_gdf['geometry'] = odk_gdf.buffer(buf_size)

print(f'After adding negative buffer of {buf_size}m : {len(odk_gdf)}')

odk_gdf['area'] = odk_gdf['geometry'].area.round(2)

odk_gdf = odk_gdf.loc[odk_gdf['area'] >= min_area]

print(f'After removing polygons where area is less than {min_area} square meters : {len(odk_gdf)}')

odk_gdf['collect'] = 'ODK_polygon'


print(f'There are {len(odk_gdf)} polygons obtained with the tablet (ODK)')
odk_gdf.head()

Initial polygons : 217
After removing duplicates ID : 217
After removing overlapping parts of polygons : 217
After adding negative buffer of -10m : 217
After removing polygons where area is less than 100 square meters : 184
There are 184 polygons obtained with the tablet (ODK)


,id,crop_code,geometry,area,collect
0,6220201174341,1.0,"POLYGON ((393906.457 1528878.301, 393894.130 1...",17964.69,ODK_polygon
1,6220201173222,13.0,"POLYGON ((394074.017 1528167.765, 394105.289 1...",764.78,ODK_polygon
2,6220201171611,1.0,"POLYGON ((394080.499 1529528.924, 394177.319 1...",8536.25,ODK_polygon
3,6220201171631,1.0,"POLYGON ((393952.711 1529569.516, 393989.334 1...",3189.76,ODK_polygon
4,6220201171661,1.0,"POLYGON ((393948.898 1529508.585, 393948.650 1...",1398.08,ODK_polygon


## 3. From ODK (tablet) - non cropland

### 3.1. Extraction from CSV file into a GeoDataFrame

In [43]:
odk_nc_df = pd.read_csv(odk_non_crop_csv)

odk_nc_df = odk_nc_df[['Picture-LC_Class','Picture-Other_Non_Cropland','start-geopoint-Latitude','start-geopoint-Longitude']]

odk_nc_df = odk_nc_df.rename(columns={"start-geopoint-Longitude": "longitude", "start-geopoint-Latitude": "latitude", "Picture-LC_Class": 'lc_code'})

#odk_nc_df = odk_nc_df.merge(lut_non_crop_df, left_on='Picture-LC_Class', right_on='lc_code')

odk_nc_gdf = gpd.GeoDataFrame(odk_nc_df, geometry=gpd.points_from_xy(odk_nc_df.longitude, odk_nc_df.latitude, crs="EPSG:4326")).to_crs(epsg=32628)

odk_nc_gdf = odk_nc_gdf.drop(columns=['longitude', 'latitude'])

odk_nc_gdf.head()

,lc_code,Picture-Other_Non_Cropland,geometry
0,5,NaN,POINT (394703.679 1527943.790)
1,5,NaN,POINT (393917.658 1529546.910)
2,5,NaN,POINT (394279.803 1529607.426)
3,99,Parcelle de mil enherbee,POINT (431783.174 1501575.154)
4,99,Parcelle de mais,POINT (431782.862 1501571.992)


### 3.2. Cleaning points

- Add positive buffer 15 meters

In [46]:
print(f'Initial polygons : {len(odk_nc_gdf)}')

odk_nc_gdf['geometry'] = odk_nc_gdf.buffer(15, resolution=24)

odk_nc_gdf['collect'] = 'ODK_point'

odk_nc_gdf.head()

Initial polygons : 80


,lc_code,Picture-Other_Non_Cropland,geometry,collect
0,5,NaN,"POLYGON ((394748.663 1527944.772, 394748.671 1...",ODK_point
1,5,NaN,"POLYGON ((393962.642 1529547.892, 393962.650 1...",ODK_point
2,5,NaN,"POLYGON ((394324.787 1529608.408, 394324.795 1...",ODK_point
3,99,Parcelle de mil enherbee,"POLYGON ((431828.157 1501576.136, 431828.165 1...",ODK_point
4,99,Parcelle de mais,"POLYGON ((431827.846 1501572.973, 431827.854 1...",ODK_point


# Load cropland data

## 1. From GPS

### 1.1. Extract GPS data

In [47]:
gps_data_df = pd.read_excel(gps_xlsx)

gps_data_df.head()

,interview__key,interview__id,id_reg,id_dep,id_arr,nom_commune,village,id_dr,id_con,id_nomCC,id_men,id_nomCM,Q1_1_1__1,Q1_1_1__2,Q1_1_1a,Q1_1_1c,Q1_1_1d,Q1_1_1d_aut,Q1_1_2a,Q1_1_2c,Q1_1_2d,Q1_1_2d_aut,consentement,Q1_1_4,Q1_1_5__Latitude,Q1_1_5__Longitude,Q1_1_5__Accuracy,Q1_1_5__Altitude,Q1_1_5__Timestamp,repondant,strate,PARCELLE__id,Q1_3a_1,Q1_3a_2,Q1_3a_3,Q1_3a_4,Q1_3a_5a,Q1_3a_5aut,Q1_3a_5c,Q1_3a_5b,Q1_3a_7,Q1_3a_8a__1,Q1_3a_8a__4,Q1_3a_8a__2,Q1_3a_8a__3,Q1_3a_8b,Q1_3a_9,Q1_3a_10a__1,Q1_3a_10a__4,Q1_3a_10a__2,Q1_3a_10a__3,Q1_3a_10b,Q1_3a_12,Q1_3a_14,Q1_3a_15,Q1_3a_16,Q1_3a_17,Q1_3a_18,Q1_4a_01,Q1_4a_02,Q1_4a_03,Q1_4a_0,Q1_4a_2,Q1_4a_3a__1,Q1_4a_3a__2,Q1_4a_3a__3,Q1_4a_3a__4,Q1_4a_3a__5,Q1_4a_3a__6,Q1_4a_3a__7,Q1_4a_3a__8,Q1_4a_3a__9,Q1_4a_3a__10,Q1_4a_7__1,Q1_4a_7__3,Q1_4a_7__4,Q1_4a_7__2,Q1_4a_7__5,Q1_4a_7__6,Q1_4a_3b,Q1_4a_4__1,Q1_4a_4__2,Q1_4a_4__3,Q1_4a_5,Q1_4a_6,sup_est,dose_semence,Q1_4a_8,Q1_4a_9a,Q1_4a_9b,Q1_4a_9c,Q1_4a_10,Q1_4a_10a,Q1_4a_10b,Q1_4a_10c,Q1_4a_11,Q1_4a_12,Q1_4a_13__1,Q1_4a_13__2,Q1_4a_13__3,Q1_4a_14,Q1_4a_15,Q1_4a_16a,Q1_4a_16b,Q1_4a_16c,Q1_4a_17a,Q1_4a_17b,Q1_4a_17c,Q1_4a_18,dateSemis,Q1_4a1_2,Q1_4a1_3a__1,Q1_4a1_3a__2,Q1_4a1_3a__3,Q1_4a1_3a__4,Q1_4a1_3a__5,Q1_4a1_3a__6,Q1_4a1_3a__7,Q1_4a1_3a__8,Q1_4a1_3a__9,Q1_4a1_3a__10,Q1_4a1_7__1,Q1_4a1_7__3,Q1_4a1_7__4,Q1_4a1_7__2,Q1_4a1_7__5,Q1_4a1_7__6,Q1_4a1_3b,Q1_4a1_4__1,Q1_4a1_4__2,Q1_4a1_4__3,Q1_4a1_5,Q1_4a1_6,sup_est1,dose_semence1,Q1_4a1_8,Q1_4a1_9a,Q1_4a1_9b,Q1_4a1_9c,Q1_4a1_10,Q1_4a1_10a,Q1_4a1_10b,Q1_4a1_10c,Q1_4a1_11,Q1_4a1_12,Q1_4a1_13__1,Q1_4a1_13__2,Q1_4a1_13__3,Q1_4a1_14,Q1_4a1_15,Q1_4a1_16a,Q1_4a1_16b,Q1_4a1_16c,Q1_4a1_17a,Q1_4a1_17b,Q1_4a1_17c,Q1_4a1_18,dateSemis1,Q1_4b_1,Q1_4b_2__1,Q1_4b_2__2,Q1_4b_2__3,Q1_4b_2__4,Q1_4b_3,Q1_4b_4__1,Q1_4b_4__2,Q1_4b_4__3,Q1_4b_5__1,Q1_4b_5__2,Q1_4b_5__3,Q1_4b_5a,Q1_4b_5b,Q1_4b_5c,Q1_4b_6__1,Q1_4b_6__2,Q1_4b_6__3,Q1_4b_6a,Q1_4b_6b,Q1_4b_6c,quant_npk,dose_npk,Q1_4b_7__1,Q1_4b_7__2,Q1_4b_7__3,Q1_4b_7a,Q1_4b_7b,Q1_4b_7c,quant_uree,dose_uree,Q1_4b_8,Q1_4b_8a__1,Q1_4b_8a__2,Q1_4b_8b,quant_npk_tot,dose_npk_epan,Q1_4b_8c,quant_uree_tot,dose_uree_epan,Q1_4b_9a,Q1_4b_9b__1,Q1_4b_9b__2,Q1_4b_9b__3,Q1_4b_9b__5,Q1_4b_9b__6,Q1_4b_9b__7,Q1_4b_9b__4,Q1_4b_10,Q1_4b_11__1,Q1_4b_11__2,Q1_4b_11__3,Q1_4b_11__4,Q1_4b_11__5,Q1_4b_11__6,Q1_4b_11__7,Q1_4b_11__8,Q1_4b_11__9,Q1_4b_11__10,Q1_4b_11__11,Q1_4b_11__12,Q1_4b_11__13,Q1_4b_11__14,Q1_4b_11__15,Q1_4b_11__16,Q1_4b_11__17,Q1_4b_11__18,Q1_4b_11__19,Q1_4b_11__20,Q1_4b_11__21,Q1_4b_11__22,Q1_4b_11__23,Q1_4b_11__24,Q1_4b_11__25,Q1_4b_11__26,Q1_4b_11__27,Q1_4b_11__28,Q1_4b_11__29,Q1_4b_11__30,Q1_4b_11__31,Q1_4b_11__32,Q1_4b_11__33,Q1_4c_1,Q1_4c_2,Q1_4c_3__1,Q1_4c_3__2,Q1_4c_3__3,Q1_4c_3__4,Q1_4c_3__5,Q1_4c_3__6,Q1_4c_4__1,Q1_4c_4__2,Q1_4c_4__3,Q1_4c_4__4,Q1_4c_4__5,Q1_4c_5__1,Q1_4c_5__2,Q1_4c_5__3,Q1_4c_5__4,Q1_4c_5__5,Q1_4c_5__7,Q1_4c_5__8,Q1_4c_5__9,Q1_4c_5__10,Q1_4c_5__11,Q1_4c_5__12,Q1_4c_5__14,Q1_4c_5__15,Q1_4c_5__13,Q1_4d_1__1,Q1_4d_1__2,Q1_4d_1__3,Q1_4d_2__1,Q1_4d_2__2,Q1_4d_2__3,Q1_4d_3__1,Q1_4d_3__2,Q1_4d_3__3,Q1_4d_4__1,Q1_4d_4__2,Q1_4d_4__3,Q1_5_0__Latitude,Q1_5_0__Longitude,Q1_5_0__Accuracy,Q1_5_0__Altitude,Q1_5_0__Timestamp,Q1_5_1,Q1_5_2a,sup_ha,Q1_5_2b,Q1_6_2c,idTraceParcelle,rendement1,rendement2,rendement3,rendement4,rendement5,rendement6,rendement7,id_com,Q1_2_1__0,Q1_2_1__1,Q1_2_1__2,Q1_2_1__3,Q1_2_1__4,Q1_2_1__5,Q1_2_1__6,Q1_2_1__7,Q1_2_1__8,Q1_2_1__9,Q1_2_1__10,Q1_2_1__11,Q1_2_1__12,Q1_2_1__13,Q1_2_1__14,Q1_2_1__15,Q1_2_1__16,Q1_2_1__17,Q1_2_1__18,Q1_2_1__19,Q1_2_1__20,Q1_2_1__21,Q1_2_1__22,Q1_2_1__23,Q1_2_1__24,Q1_2_1__25,Q1_2_1__26,Q1_2_1__27,Q1_2_1__28,Q1_2_1__29,Q1_2_1__30,Q1_2_1__31,Q1_2_1__32,Q1_2_1__33,Q1_2_1__34,Q1_2_1__35,Q1_2_1__36,Q1_2_1__37,Q1_2_1__38,Q1_2_1__39,Q1_2_1__40,Q1_2_1__41,Q1_2_1__42,Q1_2_1__43,Q1_2_1__44,Q1_2_1__45,Q1_2_1__46,Q1_2_1__47,Q1_2_1__48,Q1_2_1__49,Q1_2_1__50,Q1_2_1__51,Q1_2_1__52,Q1_2_1__53,Q1_2_1__54,Q1_2_1__55,Q1_2_1__56,Q1_2_1__57,Q1_2_1__58,Q1_2_1__59,Q1_2_10,Q1_3_0a,Q1_3a_0a,Q1_3a_1__0,Q1_3a_1__1,Q1_3a_1__2,Q1_3a_1__3,Q1_3a_

### 1.2. Clean GPS data

In [49]:
print(f'Initial rows {len(gps_data_df)}')

gps_data_subset_df = gps_data_df[['idTraceParcelle','Q1_3a_14']]

gps_data_subset_df = gps_data_subset_df.rename(columns={"idTraceParcelle": "id", "Q1_3a_14": "crop_name_2021"})

gps_data_subset_df = gps_data_subset_df.dropna(subset=['id'])

gps_data_subset_df['id'] = gps_data_subset_df['id'].str.replace('_', '')

print(f'After removing rows without ID : {len(gps_data_subset_df)}')

gps_data_subset_df = gps_data_subset_df.drop_duplicates(subset=['id'])

print(f'After removing duplicates ID : {len(gps_data_subset_df)}')

gps_data_subset_df.head()

Initial rows 396
After removing rows without ID : 367
After removing duplicates ID : 367


,id,crop_name_2021
0,6220205110042,Mil
1,6220205110041,Arachide
2,62201023262102,Arachide
3,62201023262101,Arachide
4,622020673414,Arachide


## 2. From ODK

### 2.1. Extract ODK data

In [50]:
odk_data_df = pd.read_csv(odk_crop_csv)

odk_data_df.head()

,SubmissionDate,start,end,today,start-geopoint-Latitude,start-geopoint-Longitude,start-geopoint-Altitude,start-geopoint-Accuracy,deviceid,id-Commune,id-Commune_autre,id-District,id-District_autre,id-Concession,id-Concession_autre,id-Menage,id-Mengae_autre,id-Parcelle,id-Parcelle_autre,Informations_champ-Melange,Informations_champ-Culture,Informations_champ-Culture_autre,Informations_champ-Culture2,Informations_champ-Culture_autre2,Informations_champ-Culture3,Informations_champ-Culture_autre3,Informations_champ-Proportion,Informations_champ-Homogene,Informations_champ-Photo,Informations_champ-Herbe,Informations_champ-polygone,Informations_champ-Surface,Informations_champ-Surface_arrondie,Informations_champ-Surface_note,Informations_rendement-Carre,Informations_rendement-YorN,Informations_rendement-Coordonee_carre-Latitude,Informations_rendement-Coordonee_carre-Longitude,Informations_rendement-Coordonee_carre-Altitude,Informations_rendement-Coordonee_carre-Accuracy,OTH-Comment,meta-instanceID,meta-instanceName,KEY,SubmitterID,SubmitterName,AttachmentsPresent,AttachmentsExpected,Status
0,2021-10-12T19:51:40.104Z,2021-10-12T12:08:56.064Z,2021-10-12T12:38:59.248Z,2021-10-12,13.827786,-15.981426,66.243914,3.900,collect:9HvT9ZeCXOGsnOVw,6220201,NaN,17,NaN,43,NaN,4,NaN,1,NaN,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2,1634041675046.jpg,3,13.8276875 -15.9814505 69.37716432114439 3.9;1...,31297.986337,31297.99,NaN,1,1.0,13.819180,-15.974264,62.706478,4.744,NaN,uuid:8433fb4e-0d9d-479c-a604-b0d3066ec33d,6220201_17_ 43_4_ 1,uuid:8433fb4e-0d9d-479c-a604-b0d3066ec33d,30,Enumerateurs,1,1,NaN
1,2021-10-12T19:51:15.660Z,2021-10-12T11:50:01.405Z,2021-10-12T12:02:06.382Z,2021-10-12,13.822593,-15.979953,73.553047,4.288,collect:9HvT9ZeCXOGsnOVw,6220201,NaN,17,NaN,32,NaN,2,NaN,2,NaN,1,13.0,NaN,NaN,NaN,NaN,NaN,NaN,2,1634039493488.jpg,1,13.822577 -15.9799335 67.41091576298375 3.9;13...,9618.217668,9618.22,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,uuid:6f991332-941a-42d1-b28a-1fcd04ada9a5,6220201_17_ 32_2_ 2,uuid:6f991332-941a-42d1-b28a-1fcd04ada9a5,30,Enumerateurs,1,1,NaN
2,2021-10-12T19:50:43.872Z,2021-10-12T09:43:55.438Z,2021-10-12T09:52:04.052Z,2021-10-12,13.833761,-15.981307,84.705179,3.216,collect:9HvT9ZeCXOGsnOVw,6220201,NaN,17,NaN,16,NaN,1,NaN,1,NaN,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1,13.8336799 -15.9802732 76.74953693533989 3.9;1...,17649.581646,17649.58,NaN,1,1.0,13.833566,-15.980069,80.795570,4.744,NaN,uuid:d9279760-13e0-439b-9635-9be4e64462c5,6220201_17_ 16_1_ 1,uuid:d9279760-13e0-439b-9635-9be4e64462c5,30,Enumerateurs,0,0,NaN
3,2021-10-12T19:50:42.865Z,2021-10-12T09:26:34.442Z,2021-10-12T09:36:32.167Z,2021-10-12,13.833582,-15.981483,68.465840,4.288,collect:9HvT9ZeCXOGsnOVw,6220201,NaN,17,NaN,16,NaN,3,NaN,1,NaN,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1,13.83367 -15.9814088 71.66053131205726 4.094; ...,10117.531243,10117.53,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,uuid:962af281-a512-426e-9f74-5a559150e606,6220201_17_ 16_3_ 1,uuid:962af281-a512-426e-9f74-5a559150e606,30,Enumerateurs,0,0,NaN
4,2021-10-12T19:50:33.692Z,2021-10-12T09:08:30.660Z,2021-10-12T09:19:20.190Z,2021-10-12,13.833103,-15.981382,92.066825,6.432,collect:9HvT9ZeCXOGsnOVw,6220201,NaN,17,NaN,16,NaN,6,NaN,1,NaN,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2,1634030342205.jpg,2,13.833112 -15.9814736 71.84676856254272 5.36;1...,7216.510582,7216.51,NaN,1,1.0,13.833248,-15.981236,66.273695,4.288,NaN,uuid:02e7381a-f11b-4e41-a1f4-1f0984342615,6220201_17_ 16_6_ 1,uuid:02e7381a-f11b-4e41-a1f4-1f0984342615,30,Enumerateurs,1,1,NaN


In [51]:

odk_data_df['id'] = odk_data_df['meta-instanceName'].str.replace('_', '').str.replace(' ', '')


odk_data_df = odk_data_df[['id','Informations_champ-Melange','Informations_champ-Culture','Informations_champ-Surface']]

odk_data_df['Informations_champ-Surface'] = odk_data_df['Informations_champ-Surface'].round(2)


odk_data_df = odk_data_df.merge(lut_crop_df, left_on='Informations_champ-Culture', right_on='crop_code')


odk_data_df = odk_data_df.drop_duplicates(subset=['id'])

print(f'There are {len(odk_data_df)} rows in the ODK csv.')

odk_data_df.head()

NameError: name 'lut_crop_df' is not defined

## Merge GPS tracks with GPS database

In [76]:
gps_crops_all_gdf = gps_gdf.merge(gps_crop_subset_df, left_on='id', right_on='idTraceParcelle', how='left')


print(f'There are {len(gps_crops_all_gdf.dropna(subset=["idTraceParcelle"]))} GPS tracks link to a row in the DB (equivalent to inner join)')

gps_crops_all_gdf


There are 316 GPS tracks link to a row in the DB (equivalent to inner join)


,filename,id,geometry,area,collect,idTraceParcelle,Q1_3a_14,Q1_4a_03
0,Piste_6220102-2-4-1-1.gpx,62201022411,"POLYGON ((429548.498 1516286.092, 429548.825 1...",8842.45,GPS,62201022411,Arachide,Culture pure
1,Piste_6220102-2-4-1-2.gpx,62201022412,"POLYGON ((429861.988 1516201.352, 429868.486 1...",17432.46,GPS,62201022412,Mil,Culture pure
2,Piste_6220102-2-4-1-3.gpx,62201022413,"POLYGON ((429972.245 1516435.077, 429973.528 1...",9808.82,GPS,62201022413,Mil,Culture pure
3,Piste_6220102-2-4-1-4.gpx,62201022414,"POLYGON ((429976.068 1516288.919, 429984.622 1...",5231.29,GPS,62201022414,Mais,Culture pure
4,Piste_6220102-2-16-4.gpx,62201022164,"POLYGON ((429808.854 1515355.640, 429808.571 1...",4267.04,GPS,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
329,Piste_6220301-1-12-4-2.gpx,622030111242,"POLYGON ((396282.719 1520385.249, 396281.353 1...",11195.01,GPS,622030111242,Mil,Culture pure
330,Piste_6220301-19-23-4-1.gpx,6220301192341,"POLYGON ((404005.311 1511682.508, 404006.733 1...",14934.15,GPS,6220301192341,Arachide,Culture pure
331,Piste_6220301-19-23-4-2.gpx,6220301192342,"POLYGON ((404133.363 1511763.307, 404134.054 1...",11011.16,GPS,6220301192342,Mil,Culture pure
332,Piste_6220301-19-23-4-3.gpx,6220301192343,"MULTIPOLYGON (((403709.722 1511651.014, 403683...",7959.90,GPS,6220301192343,Mil,Culture pure


# Merge ODK data with ODK geometries

In [13]:
odk_with_data_gdf = odk_gdf.merge(odk_crop_df, on='id')

print(f'There are {len(odk_with_data_gdf)} polygons obtained with the tablet (ODK)')

odk_with_data_gdf.head()

There are 215 polygons obtained with the tablet (ODK)


,id,geometry,area,collect,Informations_champ-Melange,Informations_champ-Culture,Informations_champ-Surface,crop_code,crop_name
0,6220201174341,"POLYGON ((393917.866 1528882.489, 393901.785 1...",24112.55,ODK_polygon,1,1.0,31297.99,1,Arachide
1,6220201173222,"POLYGON ((394098.110 1528308.033, 394129.216 1...",4709.83,ODK_polygon,1,13.0,9618.22,13,Pastèque
2,6220201171611,"POLYGON ((394069.867 1529538.943, 394187.774 1...",12635.46,ODK_polygon,1,1.0,17649.58,1,Arachide
3,6220201171631,"POLYGON ((393943.188 1529558.847, 393938.597 1...",6242.19,ODK_polygon,1,1.0,10117.53,1,Arachide
4,6220201171661,"POLYGON ((393939.067 1529497.632, 393938.507 1...",3883.07,ODK_polygon,1,1.0,7216.51,1,Arachide


# Merge ODK data with GPS geometries

In [25]:
gps_with_data_gdf = gps_gdf.merge(odk_crop_df, on='id', how='left')

gps_with_data_gdf['gid'] = np.arange(gps_with_data_gdf.shape[0])

print(f'There are {len(gps_with_data_gdf)} polygons obtained with GPS')

gps_with_data_gdf.head()

NameError: name 'odk_crop_df' is not defined

# Concatenate ODK polygons / ODK points / GPS

In [33]:
gdf_list = [odk_with_data_gdf, odk_non_crop_gdf, gps_with_data_gdf]

full_polygons_gdf = pd.concat(gdf_list, axis=0, ignore_index=True)

full_polygons_gdf['gid'] = np.arange(full_polygons_gdf.shape[0])

full_polygons_gdf = full_polygons_gdf.to_crs(epsg=32628)

full_polygons_gdf.head()


,id,geometry,area,collect,Informations_champ-Melange,Informations_champ-Culture,Informations_champ-Surface,crop_code,crop_name,Picture-LC_Class,Picture-Other_Non_Cropland,lc_code,lc_name,filename,gid
0,6220201174341,"POLYGON ((393929.276 1528886.677, 393909.441 1...",31096.39,ODK_polygon,1.0,1.0,31297.99,1.0,Arachide,NaN,NaN,NaN,NaN,NaN,0
1,6220201173222,"POLYGON ((394090.930 1528320.745, 394142.002 1...",9556.25,ODK_polygon,1.0,13.0,9618.22,13.0,Pastèque,NaN,NaN,NaN,NaN,NaN,1
2,6220201171611,"POLYGON ((394059.235 1529548.963, 394198.230 1...",17535.89,ODK_polygon,1.0,1.0,17649.58,1.0,Arachide,NaN,NaN,NaN,NaN,NaN,2
3,6220201171631,"POLYGON ((393936.493 1529548.371, 393926.686 1...",10052.36,ODK_polygon,1.0,1.0,10117.53,1.0,Arachide,NaN,NaN,NaN,NaN,NaN,3
4,6220201171661,"POLYGON ((393929.236 1529486.680, 393928.363 1...",7170.03,ODK_polygon,1.0,1.0,7216.51,1.0,Arachide,NaN,NaN,NaN,NaN,NaN,4


# Save to shapefile

In [25]:
odk_with_data_gdf.to_file(filename=odk_crop_shp_output)

odk_non_crop_gdf.to_file(filename=odk_non_crop_shp_output)

gps_with_data_gdf.to_file(filename=gps_shp_output)

#full_polygons_gdf.to_file(filename=full_shp_output, driver='ESRI Shapefile')

# Interactive plot with `folium`

https://leafletjs.com/reference-1.6.0.html#path-option

https://python-visualization.github.io/folium/quickstart.html

https://geopandas.org/gallery/polygon_plotting_with_folium.html

https://bikeshbade.com.np/tutorials/Detail/?title=Beginner+guide+to+python+Folium+module+to+integrate+google+earth+engine&code=8

https://nbviewer.org/github/python-visualization/folium/blob/master/examples/ImageOverlay.ipynb

In [23]:
#f = folium.Figure(width=1000, height=700)

m = folium.Map(location = [13.743747099563299, -15.772308355932424], zoom_start=11.5)#.add_to(f)

basemap_dict = folium_def.get_basemap()

basemap_dict['Google Satellite'].add_to(m)

# Plot ROI

#roi_gdf = gpd.read_file(roi_shp)

#sim_geo = gpd.GeoSeries(roi_gdf['geometry']).simplify(tolerance=0.001)
#geo_j = sim_geo.to_json()
#geo_j = folium.GeoJson(data=geo_j,
#                       style_function=lambda x: {'fillOpacity': 0, 'color': 'black'})
#geo_j.add_to(m)

# Plot GPX polygons in red

for _, r in gps_with_data_gdf.to_crs(epsg=4326).iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillOpacity': 0, 'color': 'red'})
    
    html = f'''<b>GPS</b><br>
    ID : {r['id']}<br>
    Crop type : {r['crop_code']} - {r['crop_name']}<br>
    Area : {r['area']}
    '''
    
    iframe = folium.IFrame(html, width=250, height=150)
    folium.Popup(iframe).add_to(geo_j)
    #folium.Popup('crop type ' + str(r['Informations_champ-Culture'])).add_to(geo_j)
    geo_j.add_to(m)

# Plot ODK cropland polygons
# --------------------------

for _, r in odk_with_data_gdf.to_crs(epsg=4326).iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillOpacity': 0, 'color': 'magenta'})

    html = f'''<b>ODK - Cropland</b><br>
    ID : {r['id']}<br>
    Crop type : {r['crop_code']} - {r['crop_name']}<br>
    Area : {r['area']}<br>
    Surface ODK : {r['Informations_champ-Surface']}
    '''
    
    iframe = folium.IFrame(html, width=250, height=150)
    folium.Popup(iframe).add_to(geo_j)
    geo_j.add_to(m)

# Plot ODK non-cropland points
# ----------------------------

for _, r in odk_non_crop_gdf.to_crs(epsg=4326).iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])#.simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillOpacity': 0, 'color': 'blue'})

    html = f'''<b>ODK - Non cropland</b><br>
    Land Cover : {r['lc_code']} - {r['lc_name']}<br>
    Comment : {r['Picture-Other_Non_Cropland']}<br>
    '''
    
    iframe = folium.IFrame(html, width=250, height=150)
    folium.Popup(iframe).add_to(geo_j)
    geo_j.add_to(m)



m = folium_def.add_categorical_legend(m, 'Legend',
                             colors = ['red','magenta','blue'],
                             labels = ['GPX', 'ODK - cropland','ODK - non cropland'])


m.save(map_html_output)

m

KeyError: 'area'